In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simphony.utils import dict_to_matrix
import numpy as np
from simphony.libraries import ideal
from simphony.libraries import siepic
import sax.utils as utils

/Users/noah/envs/sax/lib/python3.11/site-packages/sax/backends/__init__.py:24: UserWarning: klujax not found. Please install klujax for better performance during circuit evaluation!
  warnings.warn(


In [3]:
coupler = ideal.coupler()
display(coupler)
wl = np.linspace(1.5, 1.6, 100)
print(dict_to_matrix(siepic.waveguide(wl=wl)))

{('o2', 'o3'): 0.7071067811865476,
 ('o2', 'o1'): 0.7071067811865476j,
 ('o0', 'o3'): 0.7071067811865476j,
 ('o0', 'o1'): 0.7071067811865476,
 ('o3', 'o2'): 0.7071067811865476,
 ('o1', 'o2'): 0.7071067811865476j,
 ('o3', 'o0'): 0.7071067811865476j,
 ('o1', 'o0'): 0.7071067811865476}

[[[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.+0.j]
  [1.+0.j 0.+0.j]]

 [[0.+0.j 1.

/Users/noah/Documents/CamachoLab/simphony/simphony/libraries/siepic/models.py:974: UserWarning: Explicitly requested dtype <class 'numpy.complex128'> requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  ("o0", "o0"): jnp.zeros(wl_m.shape, dtype=np.complex128),
/Users/noah/Documents/CamachoLab/simphony/simphony/libraries/siepic/models.py:977: UserWarning: Explicitly requested dtype <class 'numpy.complex128'> requested in zeros is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  ("o1", "o1"): jnp.zeros(wl_m.shape, dtype=np.complex128),


In [4]:
dict_to_matrix(siepic.waveguide(wl=wl)).shape

(100, 2, 2)

In [5]:
S = siepic.waveguide(wl=wl)

In [6]:
arr = list(S.values())[0]
len_r = np.asarray(arr).reshape(-1)

In [7]:
len_r

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex64)

In [8]:
arr

Array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex64)

In [9]:
matrix = jnp.zeros(
        (len_r.shape, int(len(ports)), int(len(ports))), dtype=complex
    )
    for k, v in dictionary.items():
        # get index of first port
        i = ports.index(k[0])
        # get index of the second port
        j = ports.index(k[1])
        # set the value in the matrix at the i,j index to the value
        matrix = matrix.at[:, i, j].set(v)
    return matrix

IndentationError: unexpected indent (2201633092.py, line 4)